In [5]:
import torch
import torch.nn as nn
from torch import optim as opt
import torch.nn.functional as F

# 全连接层

In [22]:
w = torch.randn(5, 8, requires_grad=True)
b = torch.randn(8, requires_grad=True)
x = torch.randn(1,5)
Y = torch.randn(1,8)

In [23]:
y = F.relu(torch.matmul(x,w)+ b)
# print(x)
# print(w)
# print(b)
print(y)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 1.1611, 0.0000, 0.1851, 0.0000]],
       grad_fn=<ReluBackward0>)


In [24]:
loss = F.cross_entropy(y,Y)
loss.backward() ## 求解w和b上的梯度，但是还没有更新参数

In [28]:
print(w.grad) ## w的梯度
lr = 0.01
w_0 = w - lr * w.grad ## 更新参数

tensor([[-0.0000, -0.0000, -0.0000, -0.0000, -0.0758, -0.0000, -0.1340, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000, -0.1535, -0.0000, -0.2714, -0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0511,  0.0000,  0.0902,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0214,  0.0000,  0.0379,  0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000, -0.0926, -0.0000, -0.1637, -0.0000]])


# 训练

In [ ]:
for epoch in epochs:
    for inputs, labels in dataset:
        y = model(inputs)
        loss = F.cross_entropy(y, labels)
        loss.backward() ## 求解梯度
        opt.step() ## 优化器更新参数
        opt.zero_grad() ## 梯度清0，否则会累加

# 搭建模型

In [30]:
class CNNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.MaxPool1 = nn.MaxPool2d(kernel_size=2) # 窗口大小为2，步长也为2  [48,48] -> [24,24]
        self.MaxPool2 = nn.MaxPool2d(kernel_size=2) # [24, 24] -> [12, 12]
        self.fc1 = nn.Linear(12*12*32, 100)
        self.fc2 = nn.Linear(100, 7)
        self.bn1 = nn.BatchNorm2d(16) ## 对通道进行归一化
        self.bn2 = nn.BatchNorm2d(32)

        # self.lstm = nn.LSTM(input_size=, hidden_size=)
        # self.emb = nn.Embedding()
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.MaxPool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.MaxPool2(x)
        x = x.view(-1, 12*12*32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 数据处理

In [35]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms ## 对数据进行变形

In [36]:
transform = transforms.Compose([
    transforms.ToTensor(), ## 将图片数据转为tensor类型
    transforms.Normalize(mean=(0.5,), std=(0.5))
])

In [37]:
dataset = ImageFolder('file/mood_train/', transform = transform) ## 从路径中读取数据的同时进行数据处理(transform)
dataloader = DataLoader(dataset=dataset, batch_size=32, shuffle=True)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'file/mood_train/'

# 跑模型

In [41]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cup')
print(device)

cuda:0


In [49]:
import torch.optim as optim

model = CNNNet().to(device)
opt = optim.Adam(model.parameters(), lr=1e-3)

epochs = 8
for epoch in range(epochs):
    for i, data in enumerate(dataloader,0):
        inputs, labels = data[0].to(device), data[1].to(device)
        y = model(inputs)
        loss = F.cross_entropy(y, labels)
        loss.backward() ## 求解梯度
        opt.step() ## 优化器更新参数
        opt.zero_grad() ## 梯度清0，否则会累加

NameError: name 'dataloader' is not defined